In [68]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

In [69]:
from pyspark.sql import functions as fn

## Understand the Dataset

In [70]:
trials_df = spark.read.json('dataset.json', multiLine=True)  #Reading the Json

In [71]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [72]:
trials_df.printSchema() #Checking the schema

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [73]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [74]:
columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [75]:
trials_df.select(columns).show()

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [76]:
trials_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



# Cleaning of Data

In [77]:
new_trials_df = trials_df.select(columns)
new_trials_df.printSchema()


root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [108]:
new_column_names = {'ageofparticipant':'age_of_participant',
 'branch': 'clinic_branch',
 'name': 'head_clinician',
 'role': 'assistant_role',
 'drug_used':'drug_used',
 'experimentenddate':'experiment_end_date',
 'experimentstartdate':'experiment_start_date',
 'noofhourspassedatfirstreaction':'hours_passed_at_first_reaction',
 'conclusion':'conclusion',
 'sideeffectsonparticipant':'adverse_side_effect'
    
}

In [109]:
#Rename

new_trials_df = new_trials_df.withColumnsRenamed(new_column_names)

In [110]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+-------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|     experiment_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|           start_ts|
+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+-------------------+
|  count|              3586|         3586|          3586|          3586|     3586|                3586|                 3586|                          3513|         3586|               3586|               3586|
|   mean|17.507250418293363|         null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|

In [81]:
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistant_role': 'unknown'})

In [82]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|     experiment_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|
+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|  count|              3586|         3586|          3586|          3586|     3586|                3586|                 3586|                          3513|         3586|               3586|
|   mean|17.507250418293363|         null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|               null|
| stddev|2.3066401927555233|         null|   

# Data Transformation

In [84]:
new_trials_df.printSchema()

root
 |-- age_of_participant: long (nullable = true)
 |-- clinic_branch: string (nullable = true)
 |-- head_clinician: string (nullable = true)
 |-- assistant_role: string (nullable = false)
 |-- drug_used: string (nullable = true)
 |-- experiment_date: string (nullable = true)
 |-- experiment_start_date: string (nullable = true)
 |-- hours_passed_at_first_reaction: long (nullable = true)
 |-- conclusion: string (nullable = false)
 |-- adverse_side_effect: string (nullable = true)



In [85]:
new_trials_df.show(5)

+------------------+-------------+--------------+--------------+---------+---------------+---------------------+------------------------------+-------------+-------------------+
|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|experiment_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|
+------------------+-------------+--------------+--------------+---------+---------------+---------------------+------------------------------+-------------+-------------------+
|                19|      Ontario|          Saul|     therapist|  Placebo|  1619827200000|        1617235200000|                            52|BP normalized|     rashes on neck|
|                14|      Ontario|          Saul|         nurse| Naproxen|  1619827200000|        1617235200000|                            78|    Follow-up|                N/A|
|                17|      Ontario|          Saul|         nurse|  Placebo|  1619827200000|        161723520000

In [86]:
from pyspark.sql import types as dtypes

In [112]:
#Noticed experiment end date column is not properly named

new_trials_df =  new_trials_df.withColumnsRenamed({'experiment_date':'experiment_end_date'})

In [115]:
# convert datatypes to integers(longtype)
# divide by 1000
# convert from unix to datetime
new_trials_df = new_trials_df\
      .withColumn('start_ts', fn.from_unixtime(fn.col('experiment_start_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
      .withColumn('start_ts', fn.col('start_ts').cast(dtypes.TimestampType()))\
            .withColumn('end_ts', fn.from_unixtime(fn.col('experiment_end_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
            .withColumn('end_ts', fn.col('end_ts').cast(dtypes.TimestampType()))

In [116]:
new_trials_df.show(5)

+------------------+-------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+-------------+-------------------+-------------------+-------------------+
|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|           start_ts|             end_ts|
+------------------+-------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+-------------+-------------------+-------------------+-------------------+
|                19|      Ontario|          Saul|     therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|     rashes on neck|2021-04-01 01:00:00|2021-05-01 01:00:00|
|                14|      Ontario|          Saul|         nurse| Naproxen|      1619827200000|        1617235200

# Transformation of Data

In [129]:
#Save to csv

reordered_columns = ['start_ts', 'end_ts','clinic_branch', 'head_clinician', 'assistants_role', 'drug_used', 'age_of_participant', 'hours_passed_at_first_reaction', 'conclusion', 'observed_side_effect' ]
new_trials_df.select(reordered_columns)\
    .sort('start_ts').write.option('header', True).partitionBy('drug_used').mode("overwrite").format("csv").save('clinician')

In [135]:
#extract month and partition data for mlengineers

new_trials_df.select(reordered_columns)\
    .withColumn('start_month', fn.month('start_ts')).show(10)
        .sort('start_ts').write.partitionBy('start_month').mode("overwrite").format("parquet").save('ml_engineers')

+-------------------+-------------------+-------------+--------------+--------------+---------+------------------+------------------------------+-------------+-------------------+-----------+
|           start_ts|             end_ts|clinic_branch|head_clinician|assistant_role|drug_used|age_of_participant|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|start_month|
+-------------------+-------------------+-------------+--------------+--------------+---------+------------------+------------------------------+-------------+-------------------+-----------+
|2021-04-01 01:00:00|2021-05-01 01:00:00|      Ontario|          Saul|     therapist|  Placebo|                19|                            52|BP normalized|     rashes on neck|          4|
|2021-04-01 01:00:00|2021-05-01 01:00:00|      Ontario|          Saul|         nurse| Naproxen|                14|                            78|    Follow-up|                N/A|          4|
|2021-04-01 01:00:00|2021-05-01 01:00:00

In [133]:
new_trials_df.show(5)

+------------------+-------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+-------------+-------------------+-------------------+-------------------+
|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|           start_ts|             end_ts|
+------------------+-------------+--------------+--------------+---------+-------------------+---------------------+------------------------------+-------------+-------------------+-------------------+-------------------+
|                19|      Ontario|          Saul|     therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|     rashes on neck|2021-04-01 01:00:00|2021-05-01 01:00:00|
|                14|      Ontario|          Saul|         nurse| Naproxen|      1619827200000|        1617235200